# Natural Language Processing and the Web WS 2021/22 - Practice Class: 

# Project on Multimodal Image Processing

## Names: Luca Ciegelski, Luka Zdravkovic and Lennart Bengtson

## Preparing the coco Dataset https://cocodataset.org/#home

### Step 1: Download the dataset

To use the coco dataset funktionality of pytorch [doku here](https://pytorch.org/vision/stable/datasets.html#coco) teh coco api needs to be installed. For this run the makefile from the coco api github like this:

In [1]:
%%bash
ls coco > /dev/null || git clone https://github.com/pdollar/coco/
cd coco/PythonAPI
make
python setup.py install
cd ../..
rm -rf coco

python setup.py build_ext --inplace
running build_ext
cythoning pycocotools/_mask.pyx to pycocotools/_mask.c
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x86_64-3.9
creating build/temp.linux-x86_64-3.9/pycocotools
gcc -pthread -B /home/luka/anaconda3/compiler_compat -Wno-unused-result -Wsign-compare -DNDEBUG -O2 -Wall -fPIC -O2 -isystem /home/luka/anaconda3/include -I/home/luka/anaconda3/include -fPIC -O2 -isystem /home/luka/anaconda3/include -fPIC -I/home/luka/anaconda3/lib/python3.9/site-packages/numpy/core/include -I../common -I/home/luka/anaconda3/include/python3.9 -c ../common/maskApi.c -o build/temp.linux-x86_64-3.9/../common/maskApi.o -Wno-cpp -Wno-unused-function -std=c99
gcc -pthread -B /home/luka/anaconda3/compiler_compat -Wno-unused-result -Wsign-compare -DNDEBUG -O2 -Wall -fPIC -O2 -isystem /home/luka/anaconda3/include -I/home/luka/anaconda3/include -fPIC -O2 -isystem /home/luka/anaconda3/include -fPIC -I/home/luka/an

ls: cannot access 'coco': No such file or directory
Cloning into 'coco'...
make: Warning: File 'Makefile' has modification time 0.46 s in the future
/home/luka/anaconda3/lib/python3.9/site-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /mnt/e/projects/multimodal_image_processing_search_engine/coco/PythonAPI/pycocotools/_mask.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
../common/maskApi.c: In function ‘rleDecode’:
../common/maskApi.c:46:7: warning: this ‘for’ clause does not guard... [-Wmisleading-indentation]
   46 |       for( k=0; k<R[i].cnts[j]; k++ ) *(M++)=v; v=!v; }}
      |       ^~~
../common/maskApi.c:46:49: note: ...this statement, but the latter is misleadingly indented as if it were guarded by the ‘for’
   46 |       for( k=0; k<R[i].cnts[j]; k++ ) *(M++)=v; v=!v; }}
      |                                                 ^
../common/maskApi.c: In f

In [10]:
%%bash
mkdir -p coco_dataset/features

echo 'preparing dirs and files'
cd coco_dataset
touch photo_data.csv

echo 'downloading train2017.zip...'
aria2c -x 10 -j 10 http://images.cocodataset.org/zips/train2017.zip
echo 'downloading val2017.zip...'
aria2c -x 10 -j 10 http://images.cocodataset.org/zips/val2017.zip
echo 'unziping files'
unzip \*.zip

echo 'moving all photos into one dir'
mv train2017/ photos/
mv val2017/* photos/
rmdir val2017

echo 'downloading annotations_trainval2017.zip...'
aria2c -x 10 -j 10 http://images.cocodataset.org/annotations/annotations_trainval2017.zip
echo 'unziping files'
unzip \*.zip
rm val2017.zip
rm train2017.zip
rm annotations_trainval2017.zip

rm: cannot remove 'val2017.zip': No such file or directory
rm: cannot remove 'train2017.zip': No such file or directory


### Step 2: Process the dataset using the model

In [11]:
# imports
import os
import math
import numpy as np
import pandas as pd
import csv
import json

import torch
from PIL import Image
from IPython.display import Image as im
from transformers import CLIPProcessor, CLIPModel

In [12]:
# set the path to the dataset
cwd = os.getcwd()
dataset_path = os.path.join(cwd, "coco_dataset")

# Set the path to the photos
photos_path = os.path.join(dataset_path, "photos")

# List all JPGs in the folder
# photos_files = list(photos_path.glob("*.jpg"))
photos_files = [os.path.join(photos_path, f) for f in os.listdir(photos_path) if f.endswith('.jpg')]

# Print some statistics
print(f"Photos found: {len(photos_files)}")

Photos found: 123287


In [13]:
# Load the open CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Function that computes the feature vectors for a batch of images
def compute_clip_features(photos_batch):
    # Load all the photos from the files
    photos = [Image.open(photo_file) for photo_file in photos_batch]
    
    # Preprocess all photos
    photos_preprocessed = processor(images=photos, return_tensors='pt').to(device)
    # print(photos_preprocessed)

    with torch.no_grad():
        # Encode the photos batch to compute the feature vectors and normalize them
        photos_features = model.get_image_features(**photos_preprocessed)
        photos_features /= photos_features.norm(dim=-1, keepdim=True)

    # Transfer the feature vectors back to the CPU and convert to numpy
    return photos_features.cpu().numpy()

ftfy or spacy is not installed using BERT BasicTokenizer instead of ftfy.


In [14]:
# Define the batch size so that it fits on your GPU. You can also do the processing on the CPU, but it will be slower.
batch_size = 16

# Path where the feature vectors will be stored
features_path = os.path.join(dataset_path, "features")

# Compute how many batches are needed
batches = math.ceil(len(photos_files) / batch_size)

# Process each batch
for i in range(batches):
    print(f"Processing batch {i+1}/{batches}")

    batch_ids_path = os.path.join(features_path, f"{i:010d}.csv")
    batch_features_path = os.path.join(features_path, f"{i:010d}.npy")
    
    # Only do the processing if the batch wasn't processed yet
    if not os.path.isfile(batch_features_path):
        try:
            # Select the photos for the current batch
            batch_files = photos_files[i*batch_size : (i+1)*batch_size]

            # Compute the features and save to a numpy file
            batch_features = compute_clip_features(batch_files)
            np.save(batch_features_path, batch_features)

            # Save the photo IDs to a CSV file
            photo_ids = [os.path.splitext(os.path.basename(photo_file))[0] for photo_file in batch_files]
            photo_ids_data = pd.DataFrame(photo_ids, columns=['photo_id'])
            photo_ids_data.to_csv(batch_ids_path, index=False)
        except:
            # Catch problems with the processing to make the process more robust
            print(f'Problem with batch {i}')

Processing batch 1/7706
Processing batch 2/7706
Problem with batch 1
Processing batch 3/7706
Processing batch 4/7706
Processing batch 5/7706
Processing batch 6/7706
Processing batch 7/7706
Processing batch 8/7706
Processing batch 9/7706
Processing batch 10/7706
Processing batch 11/7706
Problem with batch 10
Processing batch 12/7706
Processing batch 13/7706
Processing batch 14/7706
Processing batch 15/7706
Processing batch 16/7706
Processing batch 17/7706
Processing batch 18/7706
Problem with batch 17
Processing batch 19/7706
Processing batch 20/7706
Processing batch 21/7706
Processing batch 22/7706
Processing batch 23/7706
Processing batch 24/7706
Processing batch 25/7706
Processing batch 26/7706
Processing batch 27/7706
Processing batch 28/7706
Processing batch 29/7706
Processing batch 30/7706
Processing batch 31/7706
Processing batch 32/7706
Processing batch 33/7706
Processing batch 34/7706
Processing batch 35/7706
Processing batch 36/7706
Processing batch 37/7706
Processing batch 38

Processing batch 308/7706
Processing batch 309/7706
Processing batch 310/7706
Processing batch 311/7706
Processing batch 312/7706
Processing batch 313/7706
Processing batch 314/7706
Processing batch 315/7706
Processing batch 316/7706
Processing batch 317/7706
Processing batch 318/7706
Processing batch 319/7706
Problem with batch 318
Processing batch 320/7706
Processing batch 321/7706
Processing batch 322/7706
Processing batch 323/7706
Processing batch 324/7706
Processing batch 325/7706
Processing batch 326/7706
Processing batch 327/7706
Processing batch 328/7706
Processing batch 329/7706
Processing batch 330/7706
Processing batch 331/7706
Processing batch 332/7706
Processing batch 333/7706
Processing batch 334/7706
Processing batch 335/7706
Processing batch 336/7706
Processing batch 337/7706
Processing batch 338/7706
Problem with batch 337
Processing batch 339/7706
Processing batch 340/7706
Processing batch 341/7706
Processing batch 342/7706
Processing batch 343/7706
Processing batch 3

Processing batch 611/7706
Processing batch 612/7706
Processing batch 613/7706
Processing batch 614/7706
Processing batch 615/7706
Processing batch 616/7706
Processing batch 617/7706
Processing batch 618/7706
Processing batch 619/7706
Processing batch 620/7706
Processing batch 621/7706
Problem with batch 620
Processing batch 622/7706
Processing batch 623/7706
Processing batch 624/7706
Processing batch 625/7706
Processing batch 626/7706
Processing batch 627/7706
Processing batch 628/7706
Processing batch 629/7706
Processing batch 630/7706
Processing batch 631/7706
Processing batch 632/7706
Processing batch 633/7706
Processing batch 634/7706
Processing batch 635/7706
Processing batch 636/7706
Processing batch 637/7706
Processing batch 638/7706
Processing batch 639/7706
Processing batch 640/7706
Processing batch 641/7706
Processing batch 642/7706
Processing batch 643/7706
Processing batch 644/7706
Processing batch 645/7706
Processing batch 646/7706
Processing batch 647/7706
Processing batc

Processing batch 918/7706
Processing batch 919/7706
Processing batch 920/7706
Processing batch 921/7706
Processing batch 922/7706
Processing batch 923/7706
Processing batch 924/7706
Processing batch 925/7706
Processing batch 926/7706
Processing batch 927/7706
Processing batch 928/7706
Processing batch 929/7706
Processing batch 930/7706
Processing batch 931/7706
Processing batch 932/7706
Processing batch 933/7706
Processing batch 934/7706
Processing batch 935/7706
Processing batch 936/7706
Processing batch 937/7706
Processing batch 938/7706
Processing batch 939/7706
Processing batch 940/7706
Processing batch 941/7706
Processing batch 942/7706
Processing batch 943/7706
Processing batch 944/7706
Processing batch 945/7706
Processing batch 946/7706
Processing batch 947/7706
Processing batch 948/7706
Processing batch 949/7706
Processing batch 950/7706
Processing batch 951/7706
Processing batch 952/7706
Processing batch 953/7706
Processing batch 954/7706
Processing batch 955/7706
Processing b

Processing batch 1218/7706
Processing batch 1219/7706
Processing batch 1220/7706
Processing batch 1221/7706
Processing batch 1222/7706
Processing batch 1223/7706
Processing batch 1224/7706
Processing batch 1225/7706
Processing batch 1226/7706
Processing batch 1227/7706
Processing batch 1228/7706
Processing batch 1229/7706
Processing batch 1230/7706
Processing batch 1231/7706
Processing batch 1232/7706
Processing batch 1233/7706
Processing batch 1234/7706
Problem with batch 1233
Processing batch 1235/7706
Processing batch 1236/7706
Processing batch 1237/7706
Processing batch 1238/7706
Processing batch 1239/7706
Processing batch 1240/7706
Processing batch 1241/7706
Processing batch 1242/7706
Processing batch 1243/7706
Processing batch 1244/7706
Processing batch 1245/7706
Processing batch 1246/7706
Processing batch 1247/7706
Processing batch 1248/7706
Processing batch 1249/7706
Processing batch 1250/7706
Processing batch 1251/7706
Processing batch 1252/7706
Processing batch 1253/7706
Proc

...

Processing batch 6252/7706
Processing batch 6253/7706
Processing batch 6254/7706
Processing batch 6255/7706
Processing batch 6256/7706
Processing batch 6257/7706
Processing batch 6258/7706
Processing batch 6259/7706
Processing batch 6260/7706
Processing batch 6261/7706
Processing batch 6262/7706
Processing batch 6263/7706
Processing batch 6264/7706
Processing batch 6265/7706
Processing batch 6266/7706
Processing batch 6267/7706
Processing batch 6268/7706
Processing batch 6269/7706
Processing batch 6270/7706
Processing batch 6271/7706
Processing batch 6272/7706
Processing batch 6273/7706
Processing batch 6274/7706
Processing batch 6275/7706
Processing batch 6276/7706
Processing batch 6277/7706
Processing batch 6278/7706
Processing batch 6279/7706
Processing batch 6280/7706
Processing batch 6281/7706
Processing batch 6282/7706
Processing batch 6283/7706
Processing batch 6284/7706
Processing batch 6285/7706
Processing batch 6286/7706
Processing batch 6287/7706
Processing batch 6288/7706
P

Problem with batch 6549
Processing batch 6551/7706
Processing batch 6552/7706
Processing batch 6553/7706
Processing batch 6554/7706
Processing batch 6555/7706
Processing batch 6556/7706
Processing batch 6557/7706
Processing batch 6558/7706
Processing batch 6559/7706
Processing batch 6560/7706
Processing batch 6561/7706
Processing batch 6562/7706
Processing batch 6563/7706
Processing batch 6564/7706
Processing batch 6565/7706
Processing batch 6566/7706
Processing batch 6567/7706
Processing batch 6568/7706
Processing batch 6569/7706
Processing batch 6570/7706
Processing batch 6571/7706
Processing batch 6572/7706
Processing batch 6573/7706
Processing batch 6574/7706
Processing batch 6575/7706
Processing batch 6576/7706
Processing batch 6577/7706
Processing batch 6578/7706
Processing batch 6579/7706
Processing batch 6580/7706
Problem with batch 6579
Processing batch 6581/7706
Processing batch 6582/7706
Processing batch 6583/7706
Processing batch 6584/7706
Processing batch 6585/7706
Process

Processing batch 6848/7706
Processing batch 6849/7706
Processing batch 6850/7706
Processing batch 6851/7706
Processing batch 6852/7706
Processing batch 6853/7706
Processing batch 6854/7706
Processing batch 6855/7706
Processing batch 6856/7706
Processing batch 6857/7706
Problem with batch 6856
Processing batch 6858/7706
Problem with batch 6857
Processing batch 6859/7706
Processing batch 6860/7706
Processing batch 6861/7706
Processing batch 6862/7706
Processing batch 6863/7706
Processing batch 6864/7706
Processing batch 6865/7706
Processing batch 6866/7706
Processing batch 6867/7706
Processing batch 6868/7706
Processing batch 6869/7706
Processing batch 6870/7706
Processing batch 6871/7706
Problem with batch 6870
Processing batch 6872/7706
Processing batch 6873/7706
Processing batch 6874/7706
Processing batch 6875/7706
Processing batch 6876/7706
Processing batch 6877/7706
Processing batch 6878/7706
Processing batch 6879/7706
Processing batch 6880/7706
Processing batch 6881/7706
Processing

Processing batch 7145/7706
Processing batch 7146/7706
Processing batch 7147/7706
Processing batch 7148/7706
Processing batch 7149/7706
Processing batch 7150/7706
Processing batch 7151/7706
Problem with batch 7150
Processing batch 7152/7706
Processing batch 7153/7706
Processing batch 7154/7706
Processing batch 7155/7706
Processing batch 7156/7706
Processing batch 7157/7706
Processing batch 7158/7706
Processing batch 7159/7706
Processing batch 7160/7706
Processing batch 7161/7706
Processing batch 7162/7706
Processing batch 7163/7706
Processing batch 7164/7706
Processing batch 7165/7706
Processing batch 7166/7706
Processing batch 7167/7706
Processing batch 7168/7706
Processing batch 7169/7706
Processing batch 7170/7706
Processing batch 7171/7706
Processing batch 7172/7706
Processing batch 7173/7706
Processing batch 7174/7706
Processing batch 7175/7706
Processing batch 7176/7706
Processing batch 7177/7706
Processing batch 7178/7706
Processing batch 7179/7706
Processing batch 7180/7706
Proc

Processing batch 7444/7706
Processing batch 7445/7706
Processing batch 7446/7706
Processing batch 7447/7706
Processing batch 7448/7706
Processing batch 7449/7706
Processing batch 7450/7706
Processing batch 7451/7706
Processing batch 7452/7706
Processing batch 7453/7706
Processing batch 7454/7706
Processing batch 7455/7706
Processing batch 7456/7706
Processing batch 7457/7706
Problem with batch 7456
Processing batch 7458/7706
Problem with batch 7457
Processing batch 7459/7706
Processing batch 7460/7706
Processing batch 7461/7706
Processing batch 7462/7706
Processing batch 7463/7706
Processing batch 7464/7706
Processing batch 7465/7706
Processing batch 7466/7706
Processing batch 7467/7706
Processing batch 7468/7706
Processing batch 7469/7706
Processing batch 7470/7706
Problem with batch 7469
Processing batch 7471/7706
Processing batch 7472/7706
Processing batch 7473/7706
Processing batch 7474/7706
Processing batch 7475/7706
Processing batch 7476/7706
Processing batch 7477/7706
Processing

In [15]:
# Load all numpy files
numpy_files = [os.path.join(features_path, f) for f in os.listdir(features_path) if f.endswith('.npy')]
features_list = [np.load(features_file) for features_file in sorted(numpy_files)]

# Concatenate the features and store in a merged file
features = np.concatenate(features_list)
np.save(os.path.join(dataset_path, "features.npy"), features)

# Load all the photo IDs
csv_files = [os.path.join(features_path, f) for f in os.listdir(features_path) if f.endswith('.csv')]
photo_ids = pd.concat([pd.read_csv(ids_file) for ids_file in sorted(csv_files)])
photo_ids.to_csv(os.path.join(dataset_path,"photo_ids.csv"), index=False)

In [16]:
photo_data_path = os.path.join(dataset_path, 'photo_data.csv')

#open the previously generated csv file
with open(photo_data_path, 'w', encoding='UTF8', newline='') as photo_csv:
    photo_data = csv.writer(photo_csv)
    header = ['id', 'url', 'description']
    photo_data.writerow(header)

    # merge val and tain dataset to get more images to search through
    for coco_anotations in [os.path.join(dataset_path, 'annotations/captions_train2017.json'),
                            os.path.join(dataset_path, 'annotations/captions_val2017.json')]:
        with open(coco_anotations) as annotations_json:
            annotations = json.load(annotations_json)
            for image in annotations['images']:
                # gets annotation for current picture
                annotation = next(anno for anno in annotations['annotations'] if anno['image_id'] == image['id'])['caption']
                # saves image data in our own photo data
                data = [image['id'], image['coco_url'], annotation]
                photo_data.writerow(data)

    # delete all files exept for images json and 

### Step 3: Cleanup
Remove all data and directories that were used to construct the feature list and photo_data.

In [17]:
%%bash
cd coco_dataset
rm -r features
rm -r annotations
rm -r photos